## Reannotates S. cerevisiae gff using empirically discovered 3'UTRs from Pelechano et al 2013. 

In [1]:
import os
import pandas as pd
import gffutils
import copy
import numpy as np
from collections import Counter

In [31]:
# Don't think I use these at all
# #these functions change the text in the attributes column after assigning a parent/child
# #relationship in a gffutils database. 
# def parent_func(parent, child):
#     #print('parent_func(%r, %r)' % (parent, child))
#     parent.attributes['child'] = child.id
    
#     return parent
    
# def child_func(parent, child):
#     #print('child_func(%r, %r)' % (parent, child))
#     child.attributes['Parent'] = parent.id
    
#     return child

In [32]:
#Load original gff data and establish basis for filenames and directory for files.
genome_assy_dir = os.path.normpath("/home/heineike/genomes/scer_20181114")
#os.path.normpath('C:\\Users\\BMH_work\\Google Drive\\UCSF\\ElSamad_Lab\\PKA\\Bioinformatics\\genome_assembly')

sc_ref_base = 'saccharomyces_cerevisiae_R64-2-1_20150113' #'scer_ref_test'
sc_ref_fn = genome_assy_dir + os.sep + sc_ref_base + '.gff'

#utr3p_fn = genome_assy_dir + os.sep + 'Nagalakshmi_2008_3UTRs_V64.gff3'#' Nag_gff_test'




## Clean up original Scer alignment file

1. By uploading via gffutils and printing again, replaces %20 with spaces, and other formatting.  Also only prints features, not sequences
2. Change chromosome name to roman numerals which work better with the genome used by bluebee (I presume that it is from Ensembl?)
3. Replace B" with Bprimeprime in YNL039W
4. For features with duplicates (which are of the following featuretypes: ['CDS', 'intron', 'noncoding_exon', 'internal_transcribed_spacer_region', 'external_transcribed_spacer_region'], replaces names with serialized version.  E.g. YLR157C-B_CDS.1 and YLR157C-B_CDS.2.   
5. For those types of features makes ID equal to the name (after making sure Names are unique in step 4).  

Reprints file as : 

saccharomyces_cerevisiae_R64-2-1_20150113_unique_ids.gff



#Note: Originally, Two lines have the exact same region and were merged when updating the database.  Changed update to key on ID or Name and that seemed to fix it. 

#around this line: 14758
#XII	SGD	external_transcribed_spacer_region	451575	451785	.	-	.	dbxref=SGD:S000029718;Parent=ETS2-1;Name=ETS2-1_external_transcribed_spacer_region
#XII	SGD	external_transcribed_spacer_region	451575	451785	.	-	.	dbxref=SGD:S000006486;Parent=RDN37-1;Name=RDN37-1_external_transcribed_spacer_region


In [464]:
#Loads the original gff as a database

#when troubleshooting you may need to close the database before remaking it. 
#orig_gff_db.conn.close()

orig_gff_db_fn = genome_assy_dir+os.sep + sc_ref_base + '.gff'

dbfn=genome_assy_dir + os.sep + sc_ref_base + '_orig.db'

orig_gff_db = gffutils.create_db(orig_gff_db_fn, dbfn=dbfn, 
                                 force=True, 
                                 #keep_order=True,
                                 #sort_attribute_values=True
                                 merge_strategy='error')

#If you don't want to recreate the database, you can load it from the file.
#orig_gff_db = gffutils.FeatureDB(dbfn) 


In [466]:
#renames all chromosomes to match the name that the SAM files from lexogen use. 

roman_numerals = ['I','II','III','IV','V','VI','VII','VIII','IX','X','XI','XII','XIII','XIV','XV','XVI']
chromosome_rename_dict = {'chr' + num : num for num in roman_numerals} 
chromosome_rename_dict['chrmt']='Mito'

for old_chr, new_chr in chromosome_rename_dict.items():
    print('Old Chromosome name: ' + old_chr + '.  New Chromosome name: ' + new_chr)
    orig_gff_db.execute("update features set seqid='{}' where seqid='{}'".format(new_chr, old_chr))

Old Chromosome name: chrXI.  New Chromosome name: XI
Old Chromosome name: chrVII.  New Chromosome name: VII
Old Chromosome name: chrI.  New Chromosome name: I
Old Chromosome name: chrIII.  New Chromosome name: III
Old Chromosome name: chrXII.  New Chromosome name: XII
Old Chromosome name: chrV.  New Chromosome name: V
Old Chromosome name: chrXIII.  New Chromosome name: XIII
Old Chromosome name: chrXVI.  New Chromosome name: XVI
Old Chromosome name: chrVI.  New Chromosome name: VI
Old Chromosome name: chrXIV.  New Chromosome name: XIV
Old Chromosome name: chrIX.  New Chromosome name: IX
Old Chromosome name: chrII.  New Chromosome name: II
Old Chromosome name: chrVIII.  New Chromosome name: VIII
Old Chromosome name: chrIV.  New Chromosome name: IV
Old Chromosome name: chrXV.  New Chromosome name: XV
Old Chromosome name: chrX.  New Chromosome name: X
Old Chromosome name: chrmt.  New Chromosome name: Mito


In [468]:
#Removed an open quote that was contained in line 18642:
#line = 'XIV	SGD	gene	555048	556832	.	+	.	ID=YNL039W;dbxref=SGD:S000004984;Name=YNL039W;Note=Essential subunit of RNA polymerase III transcription factor (TFIIIB)%3B TFIIIB is involved in transcription of genes encoding tRNAs%2C 5S rRNA%2C U6 snRNA%2C and other small RNAs;display=Essential subunit of RNA polymerase III transcription factor (TFIIIB);Ontology_term=GO:0000126,GO:0001026,GO:0001112,GO:0001156,GO:0070896,GO:0070898;orf_classification=Verified;gene=BDP1;Alias=B",BDP1,TFC5,TFC7,TFIIIB90,transcription factor TFIIIB subunit BDP1'
#YNL039W, BDP1 changed B" alias to Bprimeprime

feature = orig_gff_db['YNL039W']
att_dict = dict(feature.attributes)

new_alias_list = []
for alias in att_dict['Alias']: 
    if alias=='B"':
        new_alias_list.append('Bprimeprime')
    else: 
        new_alias_list.append(alias)

feature['Alias'] = new_alias_list

orig_gff_db.delete(feature.id, merge_strategy='error')
orig_gff_db.update([feature],
                   #id_spec = ['ID','Name'],
                   merge_strategy='error')


#Can also do line by line in the GFF
# with open(genome_assy_dir+os.sep + sc_ref_base + '_chr_rename.gff', 'r') as f:
#     lines = f.readlines()

# with open(genome_assy_dir+os.sep + sc_ref_base + '_chr_rename.gff', 'w') as f:
#     for line in lines:
#         if 'Alias=B"' in line:
#             line_split = line.split('Alias=B"')
#             line = line_split[0] + 'Alias=Bprimeprime' + line_split[1]
#         f.write(line)

In [467]:
#Print to intermediate file.  
with open(genome_assy_dir+os.sep + sc_ref_base + '_chr_rename.gff', 'w') as outfile:
    outfile.write('##gff-version 3\n')
    for feature in orig_gff_db.all_features(order_by = ('seqid','start','featuretype')):
         print(feature,file=outfile)

In [469]:
#Identifies line items with duplicate names and replaces those with serialized names


#Make list of all IDs.  If no ID, stores tuple with (No_ID, <feature_type>, <Name>)
ids = []

df = pd.read_table(genome_assy_dir+os.sep + sc_ref_base + '_chr_rename.gff', skiprows=1,header=None)

for ind,row in df.iterrows():
    feature_type = row[2]
    attribs = row[8].split(';')
    
    att_dict = {}
    id_found = False
    for att in attribs: 
        att_type,att_val = att.split('=')
        att_dict[att_type] = att_val
        if att_type == 'ID':
            ids.append(att_val)
            id_found = True
    if id_found==False:
        if 'Name' in att_dict.keys():
            ids.append(('No_ID',feature_type,att_dict['Name']))
        else: 
            ids.append( 'No ID attribute. Type = ' + feature_type + '.  Other options: ' + str(att_dict))
            raise ValueError("No ID or Name")

ids_counter = Counter(ids)

#Makes list of tuples that represent all duplicated items in the dataset.  
dupe_items = []
for item in ids_counter.items():
    if item[1]>1:
        dupe_items.append(item)

        

#Cycle through duplicated items, set Names equal to new names
print('Cleaning duplicate Names')
features_to_update = []
features_to_remove = []
for ((noid, featuretype, name),NN) in dupe_items: 
    #Finds features in database that have matching Name attributes
    cursor = orig_gff_db.execute("SELECT * FROM features WHERE featuretype='" + featuretype + "' AND attributes LIKE '%\"Name\":[\"" + name + "\"]%'")
    found_features = cursor.fetchall()
    assert len(found_features)==NN, "More features found than in duplicate items list"
    
    #sort features with matching names by start position
    id_start_list = []
    for found_feature in found_features: 
        id_start_list.append((found_feature['id'], found_feature['start']))
    id_start_list_sorted = sorted(id_start_list, key = lambda x: x[1])
    
    for jj, (old_id, start) in enumerate(id_start_list_sorted): 
        found_feature = orig_gff_db[old_id]
        features_to_remove.append(orig_gff_db[old_id])
        new_name = found_feature['Name'][0] + '.{}'.format(jj+1) 
        #found_feature['ID']=new_name  all features of selected types will have ID set to name next
        found_feature['Name']=new_name
        features_to_update.append(found_feature)
    #orig_gff_db.conn.commit()

orig_gff_db.delete(features_to_remove, merge_strategy='error')
orig_gff_db.update(features_to_update, merge_strategy='error', id_spec = ['ID','Name'])
    
print('Duplicated Names cleaned')

#cycle through all featuretypes that had duplicate Names assigned, assign the Name field as the ID
#Could do this for all fields that don't have an ID. 

print('Assigning ID as name for the following featuretypes:')

featuretypes_to_assign_ids = ['CDS', 'intron', 'noncoding_exon', 'internal_transcribed_spacer_region', 'external_transcribed_spacer_region']

print(featuretypes_to_assign_ids)


features_to_update = []
features_to_remove = []
for featuretype in featuretypes_to_assign_ids: 
    for feature in orig_gff_db.features_of_type(featuretype):
        features_to_remove.append(orig_gff_db[feature.id])
        feature['ID']=feature['Name'] 
        features_to_update.append(feature)

orig_gff_db.delete(features_to_remove, merge_strategy='error')
orig_gff_db.update(features_to_update, merge_strategy='error', id_spec = ['ID','Name'])

print('IDs assigned to features')


Cleaning duplicate Names
Duplicated Names cleaned
Assigning ID as name for the following featuretypes:
['CDS', 'intron', 'noncoding_exon', 'internal_transcribed_spacer_region', 'external_transcribed_spacer_region']
IDs assigned to features


In [470]:
#Print to file
with open(genome_assy_dir+os.sep + sc_ref_base + '_unique_ids.gff', 'w') as outfile:
    outfile.write('##gff-version 3\n')
    for feature in orig_gff_db.all_features(order_by = ('seqid','start','featuretype')):
         print(feature,file=outfile)

In [472]:
#If the file looks good, close and commit
orig_gff_db.conn.commit()
orig_gff_db.conn.close()

## Add Transcript annotation to SGD annotation file. 

In [645]:
#For metadata at the beginning of the file: 
#Note=max extent of longest transcripts in both ypd and gal
#Note=Difference between CDS and maximum of longest transcripts



#Concatenate sgd GFF with longest transcript gff from gal and ypd

#Assign transcripts as transcripts, children to the appropriate parent gene

#Assign ID for long transcript as follows: 
#ID=genename_trans.long.cond.N
#cond is gal or ypd, and number is taken from the original file eg id001
#Note=gal count X

#obtain coordinates of extent of both ypd and gal transcrips for a particular gene

#make a new transcript: 
#source column: heineike_2020
#in description column: 
#ID=genename_trans.long.max



#Use that to extend the coordinates of genes


#Subtract existing coordinates of the gene to build 

#source: heineike_2020
#three_prime_utr
#ID=genename_3pUTR.long.max or genename_5pUTR.long.max
#child of transcript

#output this file as an intermediate step to send to SGD/etc if they don't want the combined feature. 

#Make a transcript_region that consists of three_prime_utr and CDS in order to 
#count reads falling in the three prime region.  

#source: heineike_2020
#transcript_region
#ID=genename_CDS.3pUTR
#child of gene - if Parent field is filled, that will work for htseq-count

#First ensure that no other features in the file consist of transcript regions.  


In [646]:
#merges two gtf files and outputs to new file

merged_fn = genome_assy_dir+os.sep + sc_ref_base + '_tifs.gff'

sgd_tif_max_fn = {}
for cond in ['ypd','gal']:
    sgd_tif_max_fn[cond] = genome_assy_dir + os.sep + 'SGD_pelechano_reanalysis' +os.sep +"longest_full-ORF_transcripts_" + cond + ".gff3"

with open(merged_fn, 'w') as outfile:
    with open(genome_assy_dir+os.sep + sc_ref_base + '_unique_ids.gff') as sc_ref_file:
        for line in sc_ref_file:
            outfile.write(line)
        #         #keep first line: 
#         outfile.write(sc_ref_file.readline())
#         #drop header lines
#         for jj in range(0,17):
#             #print(sc_ref_file.readline())
#             sc_ref_file.readline()
#         for line in sc_ref_file: 
#             if line[0:3]=='chr':
#                 outfile.write(line)
#             else: 
#                 break
    #add transcript data for each condition
    for cond in ['ypd','gal']:
        with open(sgd_tif_max_fn[cond]) as transcript_file:    
            #drop header lines
            for  jj in range(0,3):
                line = transcript_file.readline()
                #print(line)
            for old_line in transcript_file:
                #old_line = "chrI	SGD	primary_transcript	143550	147630	.	+	.	gal=1;ID=YAL002W_id001"

                linesp = old_line.split('\t')

                #rename chromosome
                linesp[0] = chromosome_rename_dict[linesp[0]]

                #reclassify source
                linesp[1] = 'SGD_Pelechano_2013'
                
                #reclassify type as transcript
                linesp[2] = 'transcript'
                
                

                #ID= genename_trans.long.cond.sgd_id;
                #Parent=genename;
                #Note=cond count X
                atts = linesp[8]
                atts_split = atts.split(';')
                cond, count = atts_split[0].split('=')
                genename, sgd_id = atts_split[1].split('=')[1].split('_')
                sgd_id = sgd_id.strip('\n')
                linesp[8] = 'ID={}.long.{}.{};Parent={};condition={};count={}\n'.format(genename,cond,sgd_id,genename,cond,count)

                new_line = '\t'.join(linesp)

                outfile.write(new_line)

In [647]:
#Loads the merged.gff as a database

merged_db.conn.close()    #when troubleshooting you may need to close the database before remaking it. 


merged_db = gffutils.create_db(merged_fn, dbfn=genome_assy_dir + os.sep + sc_ref_base + '_tifs.db', 
                               id_spec=('ID', 'Name'), 
                               merge_strategy='error', 
                               force=True,  #makes new database
                               #keep_order=True, see no reason to keep this
                               force_gff = True)
#,
                               #force_dialect_check = True)
                               #sort_attribute_values=True)


In [648]:
#Iterate through genes, make big transcript features

#make a new transcript: 
#source column: heineike_2020
#start and stop at the min and max of the longest transcripts for both YPD and gal.  
#ID=genename_trans.long.max
#remove Note from attributes

max_transcripts = []
N =1
for feature in merged_db.features_of_type('gene'):
    genename = feature.id
    N = N+1
    if np.mod(N,1000)==0:
        print(str(N) + ' ' + genename)
    transcript_dict = {}
    transcripts_present=False
    need_copy_for_max_transcript = True
    for gene_child in merged_db.children(genename): 
        if gene_child.featuretype=='transcript':
            transcripts_present=True
            if need_copy_for_max_transcript:
                max_transcript_base = gene_child
                need_copy_for_max_transcript=False
            assert gene_child.source == 'SGD_Pelechano_2013', 'transcript not from Pelechano 2013'
            #dictionary is keyed on condition and has tuple representing start and end coordinates of transcript
            genename_child, filt, cond, sgd_id = gene_child.id.split('.')
            assert genename_child==genename, 'Child genename not equal to genename'
            if filt=='long':
                transcript_dict[cond] = (gene_child.start, gene_child.end)


    if transcripts_present: 
        #Define new coords as max extend of coords in gal and YPD
        starts = [coords[0] for coords in transcript_dict.values()]
        ends = [coords[1] for coords in transcript_dict.values()]
        new_coords = (min(starts),max(ends))
        
        #Had to use Feature class directly because I wanted to remove the Note attribute. 
        max_transcript = gffutils.Feature(seqid=max_transcript_base.seqid, 
                                    source = 'Heineike_2020',
                                    featuretype= max_transcript_base.featuretype,
                                    start = new_coords[0],
                                    end = new_coords[1],
                                    strand = max_transcript_base.strand,
                                    frame = max_transcript_base.frame, 
                                    attributes = {'ID': [genename + '.long.max'], 
                                                  'Parent': [genename]}
                                   )  
        max_transcripts.append(max_transcript)

print('Updating Database with Max transcripts')
merged_db.update(max_transcripts, merge_strategy='error', id_spec='ID')

1000 YDL020C
2000 YER039C
3000 YHR028C
4000 YLL046C
5000 YMR276W
6000 YOR313C
Updating Database with Max transcripts


In [644]:
merged_db.conn.commit()

In [649]:
#Print to file
with open(genome_assy_dir+os.sep + sc_ref_base + '_max_transcripts.gff', 'w') as outfile:
    outfile.write('##gff-version 3\n')
    for feature in merged_db.all_features(order_by = ('seqid','start','featuretype')):
         print(feature,file=outfile)

In [650]:
#Cycles through all genes, makes 3'UTR and 5'UTR from max of difference between CDS

#source: heineike_2020
#CDS_3pUTR
#ID=genename_CDS_3pUTR.long
#child of gene - if Parent field is filled, that will work for htseq-count

features_to_add = []
N=0
for feature in merged_db.features_of_type('gene'):
    genename = feature.id
    seqid = feature.seqid
    strand = feature.strand
    
    N = N+1
    if np.mod(N,1000)==0:
        print(str(N) + ' ' + genename)

    gene_data = {'CDS': []}
    for child in merged_db.children(genename):
        if child.featuretype=='CDS':
            gene_data['CDS'].append(child)
        elif child.featuretype=='transcript' and child.id.split('.')[2]=='max':
            gene_data['max']=child
            
    if gene_data['CDS']==[]:
        raise ValueError('No CDS for ' + genename)

    CDS_low = min([cds.start for cds in gene_data['CDS']])
    CDS_high = max([cds.end for cds in gene_data['CDS']])

    if 'max' in gene_data.keys():
        if strand=='-': 
            utr3p_coords = (gene_data['max'].start,CDS_low)
            utr5p_coords = (CDS_high, gene_data['max'].end)
        elif strand=='+':
            utr3p_coords = (CDS_high, gene_data['max'].end)               
            utr5p_coords = (gene_data['max'].start,CDS_low)

        #Make 3'UTR features 
        utr3p = gffutils.Feature(seqid=seqid, 
                                 source = 'Heineike_2020',
                                 featuretype = 'three_prime_utr',
                                 start = utr3p_coords[0],
                                 end = utr3p_coords[1],
                                 strand = strand,
                                 frame = '.', 
                                 attributes = {'ID': [genename + '_3pUTR.long.max'], 
                                               'Parent': [genename], 
                                               'Length': ['{}'.format(utr3p_coords[1]-utr3p_coords[0])]  
                                              }
                                       )
        features_to_add.append(utr3p)
        
        #Make 5'UTR features 
        utr5p = gffutils.Feature(seqid=seqid, 
                                 source = 'Heineike_2020',
                                 featuretype = 'five_prime_utr',
                                 start = utr5p_coords[0],
                                 end = utr5p_coords[1],
                                 strand = strand,
                                 frame = '.', 
                                 attributes = {'ID': [genename + '_5pUTR.long.max'], 
                                               'Parent': [genename], 
                                               'length': ['{}'.format(utr5p_coords[1]-utr5p_coords[0])]  
                                              }
                                       )

        features_to_add.append(utr5p)


    #Add combined CDS/3p_UTR
    utr_flag = 'False'
    if 'max' in gene_data.keys():
        #If this is true then there will be a 3pUTR
        utr_flag ='True'
        if strand=='-':
            coords = (utr3p_coords[0], CDS_high)
        elif strand=='+':
            coords = (CDS_low, utr3p_coords[1])
    else: 
        coords = (CDS_low, CDS_high)

    cds_utr3p = gffutils.Feature(seqid=seqid, 
                             source = 'Heineike_2020',
                             featuretype = 'CDS_3pUTR',
                             start = coords[0],
                             end = coords[1],
                             strand = strand,
                             frame = '.', 
                             attributes = {'ID': [genename + '_CDS_3pUTR.long'], 
                                           'Parent': [genename], 
                                           'utr': [utr_flag]  
                                          }
                                   )
    
    features_to_add.append(cds_utr3p)  

# print('Updating Database')
# merged_db.update(max_transcripts, merge_strategy='error', id_spec='ID')

1000 YDL019C
2000 YER039C-A
3000 YHR028W-A
4000 YLL045C
5000 YMR277W
6000 YOR314W


In [651]:
print('Updating Database with UTRs and combined CDS/UTR feature')
merged_db.update(features_to_add, merge_strategy='error', id_spec='ID')

Updating Database with UTRs and combined CDS/UTR feature


In [652]:
#Print to file
with open(genome_assy_dir+os.sep + sc_ref_base + '_UTRs_pelechano_max.gff', 'w') as outfile:
    outfile.write('##gff-version 3\n')
    for feature in merged_db.all_features(order_by = ('seqid','start','featuretype')):
         print(feature,file=outfile)

In [ ]:
#merged_db.conn.close()    #when troubleshooting you may need to close the database before remaking it. 

#Makes UTRs parents
for utr_3p in merged_db.features_of_type('three_prime_UTR'):
    gene_id = utr_3p.id.split('_')[0]
    #print(gene_id)
    #print(utr_3p.id)
 
    try:
        merged_db.add_relation(gene_id,utr_3p, 1, child_func = child_func, parent_func=parent_func)
#        print(utr_3p.attributes)
    except gffutils.FeatureNotFoundError:
        print('There is no matching orf for the 3prime UTR ' + gene_id)

In [92]:
with open(genome_assy_dir+os.sep + sc_ref_base + '_tiftest.gff', 'w') as outfile:
    outfile.write('##gff-version 3\n')
    for feature in merged_db.all_features(order_by = ('seqid','start','featuretype')):
         print(feature,file=outfile)

In [12]:
#Uses merge_Nag_scer64.sh to 
#sort combined UTR and annotation file and then merges the coordinates of the UTR and previous gene to get new coordinates for gene. 
#with bedtools
merge_cmd = ['/home/heineike/github/UTR_annotation/UTR_annotation/merge_Nag_scerR64.sh',
             '/home/heineike/genomes/scer_20181114/saccharomyces_cerevisiae_R64-2-1_20150113']

os.system(' '.join(merge_cmd))


0

In [13]:
#build dict of coordinates that need to be changed
merge_table = pd.read_table(genome_assy_dir+os.sep + sc_ref_base + '_nagdata_UTRchildren_merged', header = None)

coord_change_dict = {}

for row in merge_table.iterrows():
    annotation = row[1][5]
    annotation_ids = [item.split("=")[1] for item in annotation.split(";") if item.split("=")[0]=="ID"]
    for ann_id in annotation_ids: 
        if '_' in ann_id:
            if '3UTR' == ann_id.split('_')[1]:
                gene_id = ann_id.split('_')[0]
                coord_change = {}
    
                #for some reason the start coordinate for merged items on the + strand
                #had one number subtracted in bedtools coord_change['start'] = row[1][1]+1
                if row[1][3]=='+':
                    coord_change['start'] = row[1][1]+1
                elif row[1][3]=='-':
                    coord_change['start'] = row[1][1]
                coord_change['end'] = row[1][2]
                coord_change['UTR_id'] = ann_id

                coord_change_dict[gene_id] = coord_change


#coord_change_dict

In [14]:
#load new database that is sorted from bedtools
#merged_sorted_db.conn.close()   #when troubleshooting may need to close database before reloading

merged_sorted_fn = genome_assy_dir+os.sep + sc_ref_base + '_nagdata_UTRchildren_sorted.gff'


merged_sorted_db = gffutils.create_db(merged_sorted_fn, dbfn=genome_assy_dir + os.sep + sc_ref_base + '_nagdata_UTRchildren_sorted.db', force=True, keep_order=True, 
                        merge_strategy='merge', sort_attribute_values=True)

# merged_sorted_db.schema()
# cursor = merged_sorted_db.execute("select id from features where seqid = 'I'")
# row = cursor.fetchone()

In [15]:
#renames all chromosomes to match the name that the SAM files from lexogen use. 
roman_numerals = ['I','II','III','IV','V','VI','VII','VIII','IX','X','XI','XII','XIII','XIV','XV','XVI']
chromosome_rename_dict = {'chr' + num : num for num in roman_numerals} 
chromosome_rename_dict['chrmt']='Mito'

for old_chr, new_chr in chromosome_rename_dict.items():
    print('Old Chromosome name: ' + old_chr + '.  New Chromosome name: ' + new_chr)
    merged_sorted_db.execute("update features set seqid='{}' where seqid='{}'".format(new_chr, old_chr))

Old Chromosome name: chrmt.  New Chromosome name: Mito
Old Chromosome name: chrIV.  New Chromosome name: IV
Old Chromosome name: chrXIV.  New Chromosome name: XIV
Old Chromosome name: chrXIII.  New Chromosome name: XIII
Old Chromosome name: chrVIII.  New Chromosome name: VIII
Old Chromosome name: chrXI.  New Chromosome name: XI
Old Chromosome name: chrXV.  New Chromosome name: XV
Old Chromosome name: chrX.  New Chromosome name: X
Old Chromosome name: chrI.  New Chromosome name: I
Old Chromosome name: chrIII.  New Chromosome name: III
Old Chromosome name: chrXVI.  New Chromosome name: XVI
Old Chromosome name: chrVII.  New Chromosome name: VII
Old Chromosome name: chrXII.  New Chromosome name: XII
Old Chromosome name: chrV.  New Chromosome name: V
Old Chromosome name: chrVI.  New Chromosome name: VI
Old Chromosome name: chrII.  New Chromosome name: II
Old Chromosome name: chrIX.  New Chromosome name: IX


In [16]:
#Moves start and end locations for each gene per new file
jj = 0
for gene_id, coord_change in coord_change_dict.items():
    new_start = coord_change['start']
    new_end = coord_change['end']
    #prints out update statement every 1000 iterations. 
    jj = jj + 1
    if jj==1000:
        print("update features set end={} where id = '{}'".format(new_end,gene_id))
        jj = jj-1000
    merged_sorted_db.execute("update features set start={} where id = '{}'".format(new_start,gene_id))
    merged_sorted_db.execute("update features set end={} where id = '{}'".format(new_end,gene_id))

update features set end=68527 where id = 'YNL299W'
update features set end=509280 where id = 'YMR120C'
update features set end=818438 where id = 'YOR262W'
update features set end=744305 where id = 'YOR212W'
update features set end=51002 where id = 'YBL089W'


In [17]:
#Print to file
with open(genome_assy_dir+os.sep + sc_ref_base + '_UTRs.gff', 'w') as outfile:
    outfile.write('##gff-version 3\n')
    for feature in merged_sorted_db.all_features():
         print(feature,file=outfile)

#In the backup file the child tag was at the end of many of the lines.  This most recent time I ran it, 
#the child tag was in the middle. Also the size was slightly different because the backup had windows CR LF instead of unix LF

In [25]:
#after the file was printed manually removed an open quote that was contained in line 22795:
#line = 'XIV	SGD	gene	555048	556886	.	+	.	ID=YNL039W;dbxref=SGD:S000004984;Name=YNL039W;Note=Essential subunit of RNA polymerase III transcription factor (TFIIIB)%3B TFIIIB is involved in transcription of genes encoding tRNAs%2C 5S rRNA%2C U6 snRNA%2C and other small RNAs;display=Essential subunit of RNA polymerase III transcription factor (TFIIIB);Ontology_term=GO:0000126,GO:0001026,GO:0001112,GO:0001156,GO:0070896,GO:0070898;orf_classification=Verified;child=YNL039W_3UTR;gene=BDP1;Alias=B",BDP1,TFC5,TFC7,TFIIIB90,transcription factor TFIIIB subunit BDP1'
#YNL039W, BDP1 changed B" alias to Bprimeprime

with open(genome_assy_dir+os.sep + sc_ref_base + '_UTRs.gff', 'r') as f:
    lines = f.readlines()

with open(genome_assy_dir+os.sep + sc_ref_base + '_UTRs.gff', 'w') as f:
    for line in lines:
        if 'Alias=B"' in line:
            line_split = line.split('Alias=B"')
            line = line_split[0] + 'Alias=Bprimeprime' + line_split[1]
        f.write(line)


    

In [18]:
#If the output looked good, commit and print to file
merged_sorted_db.conn.commit()

In [26]:
#some useful commands to access / change data in gffutils. 

#db_id = 'YEL058W'
#cursor = merged_db.execute('select * from features where id ="%s"' % db_id)
#row = cursor.fetchone()
#row['end']

#feat = list(merged_db.features_of_type('three_prime_UTR'))[0]


#pd.read_sql('select * from features;', merged_db.conn)
